This exercise aims to understand my spending and what areas generated a lot of revenue and what were the biggest spenders. I picked a random month, July 2021. I will be answering these questions.
a. How much did I withdraw for the month?
b. How much did I deposit?
c. What day had the most number of transactions?
d. The day with highest transaction?
e. How much have I Transferred funds to other registered users and which is the highest/lowest amount?
f. How much have I received funds from other registered users and which is the highest/lowest amount?
g. Amount Safaricom have charged using their platform?
h. How much have I spent paying bills through Mpesa?

Importing the libraries. Pandas is useful in providing easy analysis and data manipulation. Numpy allows for use of mathematical functions to arrays and matrices. Matplotlip allows for ploting and graphical presentation as well as interactive visualization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sqlalchemy.sql.expression import column

We will call our dataframe (fancy name for table in pandas/python) 'statement'. It suffices to note that since safaricom statements are downloaded in pdf format, you have to convert it into CSV file.
There are various ways to convert pdf to CSV but for this excercise, I used [Tabula](https://tabula.technology/). (Note you also have to remove password before conversion since the statement is password protected. I did this using [I love PDF](https://www.ilovepdf.com/unlock_pdf)


After importing the data in CSV format, Data cleaning starts. Some columns may be un aligned and might need some formating. Take this opportunity to check the overall appearance of data. This is possible using Excel. Open a blank excel sheet and import data from the downloaded CSV. Use excel to delete unnecessary columns (You can still do this on python, i just prefer it here) Do any other preliminary checks. SOme other cleaning I did was to remove the negative sign on withdrawals as well as ensure the two columns are numeric. Then read your csv here

In [ ]:
statement = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/mpesa_data.xlsx')

In [ ]:
statement.head(5)

,Receipt No.,Completion Time,Details,Transaction,Paid In,Withdrawn,Balance
0,PGV6ZU9JQW,7/31/2021 10:02,Pay Bill Charge,Completed,NaN,23.0,4.6
1,PGV6ZU9JQW,7/31/2021 10:02,Pay Bill Online to 888880 - KPLC PREPAID Acc. ...,Completed,NaN,150.0,27.6
2,PGU9YZXIZT,7/30/2021 17:15,Merchant Payment to 189119 - BUMA LOGISTICS LI...,Completed,NaN,120.0,177.6
3,PGU5YZS1T3,7/30/2021 17:13,Customer Transfer of Funds Charge,Completed,NaN,22.0,297.6
4,PGU5YZS1T3,7/30/2021 17:13,Customer Transfer to 254726710236 - STANLEY KO...,Completed,NaN,1350.0,319.6


A preview of the first five transactions provides a glimpse of our table. In my opinion, the columns 'Receipt No.', 'Transaction' and 'Balance' serve no purpose in our analysis. So we drop them

In [ ]:
statement.drop(columns= ["Receipt No.","Transaction", "Balance"], inplace = True)

We also need to protect the privacy of those who have sent or recieved. We therefore need to mask out the number. I found a way to do this by considering only the first two Names after the hyphen.

In [ ]:
def extract_info(details):
    match = re.search(r'-\s*([^\-]+)$', details)
    if match:
        return match.group(1)
    return details

statement.Details = statement.Details.apply(extract_info)

In [ ]:
#A quick check to ensure our data types are as we want them to be
statement.dtypes

Completion Time     object
Details             object
Paid In            float64
Withdrawn          float64
dtype: object

In [ ]:
statement.head()

,Completion Time,Details,Paid In,Withdrawn
0,7/31/2021 10:02,Pay Bill Charge,NaN,23.0
1,7/31/2021 10:02,KPLC PREPAID Acc. 54602085521,NaN,150.0
2,7/30/2021 17:15,BUMA LOGISTICS LIMITED4,NaN,120.0
3,7/30/2021 17:13,Customer Transfer of Funds Charge,NaN,22.0
4,7/30/2021 17:13,STANLEY KOSKEI,NaN,1350.0


In [ ]:
#Fill out the blank spaces with zeros
statement.fillna(0, inplace = True)

In [ ]:
statement.tail()

,Completion Time,Details,Paid In,Withdrawn
127,7/2/2021 15:23,kelvin kinyanjui\nmwaura,0.0,4100.0
128,7/2/2021 15:18,ANTHONY\nMWIRIGI,8300.0,0.0
129,7/1/2021 19:15,Customer Transfer of Funds Charge,0.0,55.0
130,7/1/2021 19:15,CHARLES\nMAKAMA KAMAU,0.0,5000.0
131,7/1/2021 13:19,Equity Bulk Account\nvia API. Original convers...,21000.0,0.0


The first question is the total withdrawal vs total paid in

In [ ]:
total_withdrawals = statement.Withdrawn.sum()
total_recieved = statement['Paid In'].sum()
summary = {'Total Withdrawals': [total_withdrawals], 'Total Paid In': [total_recieved]}
withdrawalsVSrecieved = pd.DataFrame(summary)
withdrawalsVSrecieved

,Total Withdrawals,Total Paid In
0,76994.0,78950.0


First we need to make sure the completion time column has the the object type as date

In [ ]:
statement['Completion Time'] = pd.to_datetime(statement['Completion Time'])

We can group the table on dates and get the sum of the totals per day. But first, we have to extract the date part.

In [ ]:
statement['Date'] = statement['Completion Time'].dt.date

In [ ]:
transactions_per_day = statement['Completion Time'].dt.date.value_counts()

In [ ]:
day_with_most_transactions = transactions_per_day.idxmax()
most_transactions_count = transactions_per_day.max()
print(f"The day with the most transactions is {day_with_most_transactions} with {most_transactions_count} transactions.")

The day with the most transactions is 2021-07-11 with 10 transactions.


In [ ]:
daily = statement.groupby('Date').sum()
daily.head()

<ipython-input-106-ed62892fecdd>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  daily = statement.groupby('Date').sum()


,Paid In,Withdrawn
Date,,
2021-07-01,21000.0,5055.0
2021-07-02,8300.0,4155.0
2021-07-03,7200.0,18874.0
2021-07-04,0.0,3874.0
2021-07-05,1300.0,2482.0


In [ ]:
highest_transaction_paidin = daily['Paid In'].max()
max_paidin_transaction = daily['Paid In'].idxmax()
max_paidin_transaction_date = max_paidin_transaction.strftime('%Y-%m-%d')
print(f"{max_paidin_transaction_date} recorded the highest transaction of {highest_transaction_paidin} as Paid In Amount.")

2021-07-01 recorded the highest transaction of 21000.0 as Paid In Amount.


In [ ]:
highest_transaction_Withdrawal = daily['Withdrawn'].max()
max_withdrawn_transaction = daily['Withdrawn'].idxmax()
max_withdrawn_transaction_date = max_withdrawn_transaction.strftime('%Y-%m-%d')
print(f"{max_withdrawn_transaction_date} recorded the highest transaction of {highest_transaction_Withdrawal} as Withdrawal Amount.")

2021-07-03 recorded the highest transaction of 18874.0 as Withdrawal Amount.
